In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup

In [ ]:
### Ana mantık şöyle işliyor. Son dakika haberleri gün gün ve sayfa sayfa giriliyor. En çok okunan haberlerin sayısı ise genellikle 5
### O yüzden bugünun tarihinden geriye doğru gidiyoruz. Eğer haber sayısı 5'e eşit olursa loop kırılıyor
### Böylece her gün yayınlanan haberleri buluyor bunu bir DataFrame üzerinden bir .csv dosyasına kaydediyoruz
### Sonrasında linkleri teker teker indiriyoruz.

df = pd.DataFrame()
for day in range(0,365):
    for page in range(1,10):
            ### Hangi gün hangi sayfada olduğumuzu print ediyoruz
            print(f'{(datetime.today()- timedelta(days=day)).strftime("%d-%m-%Y")} - {page}', end='\r')
            ### Url'yi oluşturuyoruz
            url = f'https://www.ekonomim.com/son-dakika-haberleri?date={(datetime.today()- timedelta(days=day)).strftime("%d-%m-%Y")}&sayfa={page}'
            ### Sayfayı indiriyoruz
            response = requests.get(url)  # url of next page
            ### Html formatına getiriyoruz
            soup = BeautifulSoup(response.content, 'html.parser')
            ### Linkleri çıkartıyoruz
            newslist = soup.find_all("div", {"class": "container category-page"})
            ### İçinde sayfa numarası olan linklerden kurtuluyoruz ve yalnızca https içerikli linkleri alıyoruz
            newslinks = []
            for item in newslist:
                for link in item.find_all('a', href=True):
                    if 'sayfa=' in link['href']: 
                        continue
                    elif 'https' in link['href']:
                        newslinks.append(link['href'])
            newslinks = list(set(newslinks))
            ### Eğer o sayfadaki link sayısı 5 ise, Çok okunanların sayısı, loop kırılıyor ve diğer tarihe geçiyoruz
            if len(newslinks)  == 5:
                break
            else: 
                a = pd.DataFrame({
                    'Tarih': (datetime.today()- timedelta(days=999)).strftime("%d-%m-%Y"),
                    'Link' : newslinks
                })
                df = pd.concat([df,a])
### Tekrar eden linklerden, Çok okunanlardan, kurtuluyoruz
df = df.drop_duplicates().reset_index(drop=True)      
### Kaydediyoruz
df.to_csv('base_news.csv', encoding='utf-8', index=False)      
  

In [ ]:
### Dosyayı Oku
df = pd.read_csv('./base_news.csv') 
### Haber linklerindeki metni indirip bir dosyaya kaydet.
### Buradaki range'i len(df) ile değiştirmek lazım
for r in range(0,100):
    ### Data Frame'deki linki al
    url = df['Link'][r]
    response = requests.get(url)  # url of next page
    soup = BeautifulSoup(response.content, 'html.parser'),
    ### Haber metnini indir ve texte dönüştür
    news = soup[0].find_all("div", {"class": "content-text"})
    news2 = news[0].text
    ### Kaydet
    f = open(f"./News/{df['Tarih'][r] + '_' +url.split('/')[len(url.split('/'))-1][:24]}.txt", "a", encoding="utf-8")
    f.write(news2)
    f.close()
    ### İşlemin yüzde kaçında olduğumuzu print ettir.
    print(round(r/100*100,2), end='\r')